In [17]:
import nibabel as nb
import numpy as np
import scipy.io
import scipy
from scipy import stats
import math
import pandas as pd
import statsmodels.api as sm

In [18]:
regressor_scene_transitions = np.load('E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_absdif_intensity_pixels/regressor_mean_maxdif_pixel_intensitiy.npy')

In [19]:
regressor_scene_transitions

array([2.96055170e-03, 1.62519290e-04, 7.23379630e-06, ...,
       0.00000000e+00, 1.44675926e-06, 0.00000000e+00])

In [20]:
np.mean(regressor_scene_transitions)

15.363017531713911

In [22]:
np.median(regressor_scene_transitions)

4.788623649691358

In [23]:
regressor_scene_transitions_binarized = (regressor_scene_transitions>np.mean(regressor_scene_transitions)).astype(int)

In [27]:
regressor_scene_transitions_binarized_shifted = np.roll(regressor_scene_transitions_binarized, 4)

In [28]:
regressor_scene_transitions_binarized.sum()

1032

In [29]:
regressors = {}
for i in range(-5, 11):
    regressors[i] = np.roll(regressor_scene_transitions_binarized_shifted, i)

In [26]:
left_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_left_Meanfile.mat')['Left_data']
right_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']

In [31]:
for regressor_num in range(-5, 11):
    corr_right = np.zeros(32492)
    pvalue_right = np.zeros(32492)
    for i in range (32492):
        corr,p_value = stats.pearsonr(regressors[regressor_num], right_data[i])
        corr_right[i] = corr
        pvalue_right[i] = p_value
        

    corr_right = np.nan_to_num(corr_right)
    corr_right_32 = corr_right.astype(np.float32)

    temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
    temp_r = nb.load(temp_r_path)
    temp_r.darrays[0].data = corr_right_32
    temp_r.to_filename(f'E:/FPP_files_and_codes/FPP_files/transition_analsysis_test/correlation_maps/correlation_map{regressor_num}.R.func.gii')

    significance_right = -1 * np.log(pvalue_right) * np.sign(corr_right)

    significance_right = np.nan_to_num(significance_right, nan=0.0, posinf=0.0, neginf=0.0)
    significance_right_32 = significance_right.astype(np.float32)

    temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
    temp_r = nb.load(temp_r_path)
    temp_r.darrays[0].data = significance_right_32
    temp_r.to_filename(f'E:/FPP_files_and_codes/FPP_files/transition_analsysis_test/significance_maps/significance_map{regressor_num}.R.func.gii')

C:\Users\roza\AppData\Local\Temp\ipykernel_16032\4087007522.py:5: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(regressors[regressor_num], right_data[i])


In [32]:
for regressor_num in range(-5, 11):
    corr_left = np.zeros(32492)
    pvalue_left = np.zeros(32492)
    for i in range (32492):
        corr,p_value = stats.pearsonr(regressors[regressor_num], left_data[i])
        corr_left[i] = corr
        pvalue_left[i] = p_value
        

    corr_left = np.nan_to_num(corr_left)
    corr_left_32 = corr_left.astype(np.float32)

    temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
    temp_l = nb.load(temp_l_path)
    temp_l.darrays[0].data = corr_left_32
    temp_l.to_filename(f'E:/FPP_files_and_codes/FPP_files/transition_analsysis_test/correlation_maps/correlation_map{regressor_num}.L.func.gii')

    significance_left = -1 * np.log(pvalue_left) * np.sign(corr_left)

    significance_left = np.nan_to_num(significance_left, nan=0.0, posinf=0.0, neginf=0.0)
    significance_left_32 = significance_left.astype(np.float32)

    temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
    temp_l = nb.load(temp_l_path)
    temp_l.darrays[0].data = significance_left_32
    temp_l.to_filename(f'E:/FPP_files_and_codes/FPP_files/transition_analsysis_test/significance_maps/significance_map{regressor_num}.L.func.gii')

C:\Users\roza\AppData\Local\Temp\ipykernel_16032\976138081.py:5: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(regressors[regressor_num], left_data[i])


### making video from maps

In [40]:
import cv2
import os

def add_subtitle_to_image(image, subtitle, font=cv2.FONT_HERSHEY_SIMPLEX, 
                          font_scale=1, font_color=(255, 255, 255), thickness=2):
    """ Adds a subtitle text to an image centered at the bottom """
    height, width, _ = image.shape
    text_size = cv2.getTextSize(subtitle, font, font_scale, thickness)[0]
    text_x = (width - text_size[0]) // 2  # Center the text horizontally
    text_y = height - 10  # Position the text at the bottom with some margin

    # Add a black rectangle background for the subtitle text for better visibility
    cv2.rectangle(image, (text_x - 10, text_y - text_size[1] - 10), 
                  (text_x + text_size[0] + 10, text_y + 10), (0, 0, 0), -1)
    
    # Add the text on the image
    cv2.putText(image, subtitle, (text_x, text_y), font, font_scale, font_color, thickness)
    return image

def create_video(image_folder, output_video_path, frame_rate, display_time_per_image, subtitles):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png") or img.endswith(".jpg")]
    

    # Read the first image to get dimensions
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    frames_per_image = int(frame_rate * display_time_per_image)

    for idx, image in enumerate(images):
        img = cv2.imread(os.path.join(image_folder, image))
        # Add subtitle to the image
        subtitle = subtitles[idx] if idx < len(subtitles) else ""
        img_with_text = add_subtitle_to_image(img, subtitle)
        for _ in range(frames_per_image):
            video.write(img_with_text)

    video.release()




In [41]:

subtitles = [f'{i}' for i in range(-5,16)]
create_video('E:/FPP_files_and_codes/FPP_files/transition_analsysis_test/maps', 'E:/FPP_files_and_codes/FPP_files/transition_analsysis_test/maps.mp4', 30, 2, subtitles)
